In [56]:
#Auxiliary notebook to run from A3_jonathan
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
n_steps_factor = 1

The file_gams_py_gdb0.gdx is still active and was not deleted.
The file_gams_py_gdb1.gdx is still active and was not deleted.
The file_gams_py_gdb2.gdx is still active and was not deleted.
The file_gams_py_gdb3.gdx is still active and was not deleted.
The file_gams_py_gdb5.gdx is still active and was not deleted.
The file_gams_py_gdb7.gdx is still active and was not deleted.


In [57]:
# catalog = "main_catalog"
# currapp = False
# os.chdir(directory['py'])
# import techdata_to_tree, sys, ShockFunction
# import abatement_EOP_new as abatement
# import ShockFunction
# from DB2Gams import OrdSet as OS
# os.chdir(directory['curr'])
# with open(os.getcwd()+'\\conopt_ExtraPivot.opt',"r") as file:
#     kwargs_mi = {'opt_file': file.read()}

In [63]:
def organize_sol_data(sols, x_var, nodes_to_save, node_to_shock, tech_catalog):
#     warnings.filterwarnings("error", UserWarning)
    list_of_dfs = []
    for var in sols:
        if var == x_var:
            continue
        df = sols[var]
        index_name = df.index.names[1]
        if len(df.index.get_level_values(index_name).unique()) == 1:
            only_one_series = True
        else:
            only_one_series = False
        df = df.reset_index()
        df2 = df.pivot_table(df.columns[-1], df.columns[0], df.columns[1]).reset_index()
        if tech_catalog in nodes_to_save and not only_one_series:
            df2 = df2[[x_var + "_loop"] + [c for c in nodes_to_save[tech_catalog] if c in df2.columns]]
        df3 = df2.merge(sols[x_var].iloc[sols[x_var].index.get_level_values(1) == node_to_shock].reset_index().drop(columns=sols[x_var].index.names[1]), \
                        how="inner", on=x_var+"_loop").drop(columns=x_var+"_loop")
        df3 = df3.rename(columns={"sol_"+x_var:x_var}).set_index(x_var).sort_values(by=x_var)
        if only_one_series:
            df3.rename(columns={df3.columns[0]:var}, inplace=True)
        else:
            df3.rename(columns={key:var + "_"+ key for key in df3.columns}, inplace=True)
        list_of_dfs.append(df3)
    return list_of_dfs

In [64]:
out = {}

#### Load and setup

In [102]:
#Load model
gm = abatement.abate(pickle_path=directory["gams"] + "/" + catalog, work_folder=directory['work'])
gm.reset_settings()
gm.add_functions()
if catalog in ["main_catalog", "categoryexperiment_EOP"]:
    gm.setstate("EOP")
elif catalog in ["categoryexperiment_ID"]:
    gm.setstate("ID")

In [103]:
name = "CO2tax"
x_var = "pM"
node_to_shock = "CO2"

In [104]:
vars_ = ["M0", "PbT", "pM", "PwThat", "qD"]
if "EOP" in gm.state:
    vars_ += ["M"]
if currapp:
    vars_ += ["muG", "sigmaG"]
store_sol = {var:{"domains":x_var + '_loop'} for var in vars_}

In [105]:
db_end_of_loop = DataBase.GPM_database()
db_end_of_loop[x_var] = gm.get(x_var).copy()
db_end_of_loop[x_var].vals[node_to_shock] = db_end_of_loop[x_var].vals[node_to_shock]*0 + 20

In [106]:
gm.write_and_run(name=name,kwargs_init={'check_variables':True},add_checkpoint=name, kwargs_mi=kwargs_mi)
gm.checkpoints[name + "_cp"] = gm.model_instances[name].ws.add_checkpoint()

In [107]:
print(gm.model_instances[name].solve_sneakily(db_star=db_end_of_loop,from_cp = True, cp_init = gm.checkpoints[name], 
                                              options_run={'checkpoint': gm.checkpoints[name + '_cp']}, 
                                              kwargs_shock={"n_steps":int(500*n_steps_factor), "loop_name":x_var + "_loop", "store_sol":store_sol}))

{'Modelstat': 16.0, 'Solvestat': 1.0}


In [111]:
mi = gm.model_instances[name]
assert mi.modelstat

In [99]:
sols = {key:mi.out_db.series["sol_" + key].vals for key in store_sol.keys()}
nodes_to_save = {
    "main_catalog":["U_ID_t2_1", "U_EOP_t1_1", "U_EOP_t2_1", "U_ID_t1_2", "C_EH_2", "U0_ID_C_EH_2", "CO2", "basetech_EH", "basetech_EH_oil", 
                    "oil", "U_ID_t1_1", "U_ID_t2_2", "U0_ID_C_EH_1", "U0_ID_C_EH_3", "ID_t1", "ID_t2", "C_CO2_1", "C_CO2_2"],
    "categoryexperiment_EOP":["U_EOP_t1_1", "CO2", "oil", "basetech_EH_oil"],
    "categoryexperiment_ID":["U_ID_t1_1", "CO2", "oil", "basetech_EH_oil", "U0_ID_C_EH_2", "C_EH_2", "basetech_EH"]
}

In [73]:
gm.model.database.merge_dbs(gm.model.database,mi.out_db,'second')

In [30]:
out[name] = organize_sol_data(sols, x_var, nodes_to_save, node_to_shock, gm.model.settings.catalog)

In [32]:
df_out = pd.concat(out[name], axis = 1)
df_out.to_pickle(x_var + "_" + gm.model.settings.catalog + ".pkl")